# LSTM for Sentiment Analysis

Followed [this tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html), and completed it [here](tut/nlp/4_lstm.ipynb)

Helpful blogs:
* [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class SentimentAnalysisModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int = 30,
        hidden_size: int = 40,
        n_rnn_layers: int = 1,
    ):
        super().__init__()
        self.emb = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
        )

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=n_rnn_layers,
        )

        self.seq = nn.Sequential(
            nn.Linear(hidden_size, 500),
            nn.ReLU(),
            nn.Linear(500, 2),
        )

    def forward(self, x: torch.Tensor):
        # x shape: (B, L)
        # convert token indices to embedding values
        x = self.emb(x)
        # x shape: (B, L, Emb dim)
        
        x = x.transpose(0, 1)

        _, x = self.lstm(x) 


        # take only the last layer
        x = x[-1, :, :]


        # x shape: (B, Hidden size?)

        return self.seq(x)# Loading 

In [ ]:
# load in tokenized data
data_dict = torch.load("data/imdb_data.pt")
data = data_dict["reviews"]
labels = data_dict["labels"]
lengths = data_dict["lengths"]

# split into train and test by 80:20
training_fraction = 0.8

train_data = data[: int(len(data) * training_fraction)]
train_labels = labels[: int(len(data) * training_fraction)]
train_lengths = lengths[: int(len(data) * training_fraction)]

test_data = data[int(len(data) * training_fraction) :]
test_labels = labels[int(len(data) * training_fraction) :]
test_lengths = lengths[int(len(data) * training_fraction) :]